競馬データの収集（スクレイピング）

In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from tqdm import tqdm


次のコードは、競馬のレース結果データをスクレイピングするクラス `Results` とその静的メソッド `scrape` を定義しています。

##### クラス `Results`
競馬のレース結果データをスクレイピングするメソッドを提供します。

##### メソッド `scrape`
与えられたレースIDリストからレース結果データをスクレイピングし、データフレームとして返します。

##### パラメータ
- `race_id_list` (list): レースIDのリスト

##### 戻り値
- `race_results_df` (pandas.DataFrame): 全レース結果データをまとめたDataFrame

##### 処理の流れ
1. `race_results` 辞書を初期化し、レースIDをキーとしてデータフレームを格納します。
2. 各レースIDについて、以下の処理を行います。
    - 1秒待機してから、指定されたURLからHTMLデータを取得します。
    - `pandas.read_html` でメインのテーブルデータを取得し、列名の半角スペースを除去します。
    - `BeautifulSoup` で天候、レースの種類、コースの長さ、馬場の状態、日付などの情報を取得し、データフレームに追加します。
    - 馬IDと騎手IDをスクレイピングし、データフレームに追加します。
    - データフレームのインデックスをレースIDに設定し、`race_results` 辞書に格納します。
3. 全てのレース結果データを一つのデータフレームに結合し、返します。


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

class Results:
    @staticmethod
    def scrape(race_id_list):
        """
        レース結果データをスクレイピングする関数
        Parameters:
        ----------
        race_id_list : list
            レースIDのリスト
        Returns:
        ----------
        race_results_df : pandas.DataFrame
            全レース結果データをまとめてDataFrame型にしたもの
        """
        # リトライ機能の設定
        session = requests.Session()
        retry = Retry(
            total=5,  # リトライ回数
            backoff_factor=1,  # リトライ間隔
            status_forcelist=[500, 502, 503, 504],  # リトライ対象のステータスコード
        )
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('https://', adapter)

        # race_idをkeyにしてDataFrame型を格納
        race_results = {}
        for race_id in tqdm(race_id_list):
            time.sleep(1)
            try:
                url = "https://db.netkeiba.com/race/" + race_id
                # スクレイピング
                html = session.get(url)
                html.encoding = "EUC-JP"
                # メインとなるテーブルデータを取得
                df = pd.read_html(html.text)[0]
                # 列名に半角スペースがあれば除去する
                df = df.rename(columns=lambda x: x.replace(' ', ''))
                # 天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
                soup = BeautifulSoup(html.text, "html.parser")
                texts = (
                    soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                    + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
                )
                info = re.findall(r'\w+', texts)
                for text in info:
                    if text in ["芝", "ダート"]:
                        df["race_type"] = [text] * len(df)
                    if "障" in text:
                        df["race_type"] = ["障害"] * len(df)
                    if "m" in text:
                        df["course_len"] = [int(re.findall(r"\d+", text)[-1])] * len(df)
                    if text in ["良", "稍重", "重", "不良"]:
                        df["ground_state"] = [text] * len(df)
                    if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                        df["weather"] = [text] * len(df)
                    if "年" in text:
                        df["date"] = [text] * len(df)
                # 馬ID、騎手IDをスクレイピング
                horse_id_list = []
                horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/horse")}
                )
                for a in horse_a_list:
                    horse_id = re.findall(r"\d+", a["href"])
                    horse_id_list.append(horse_id[0])
                jockey_id_list = []
                jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                    "a", attrs={"href": re.compile("^/jockey")}
                )
                for a in jockey_a_list:
                    jockey_id = re.findall(r"\d+", a["href"])
                    jockey_id_list.append(jockey_id[0])
                df["horse_id"] = horse_id_list
                df["jockey_id"] = jockey_id_list
                # インデックスをrace_idにする
                df.index = [race_id] * len(df)
                race_results[race_id] = df
            # 存在しないrace_idを飛ばす
            except IndexError:
                continue
            except AttributeError:  # 存在しないrace_idでAttributeErrorになるページもあるので追加
                continue
            # wifiの接続が切れた時などでも途中までのデータを返せるようにする
            except Exception as e:
                print(f"An error occurred for race_id {race_id}: {e}")
                continue
            # Jupyterで停止ボタンを押した時の対処
            except:
                break
        # pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])
        return race_results_df

```markdown
次のコードは、指定された範囲の場所、回、日、レース番号の組み合わせからレースIDを生成し、そのリストを用いてレース結果データをスクレイピングしています。
```

In [ ]:
years = range(2020, 2024)  # 例: 2020年から2023年まで


for year in years:
    race_id_list = []
    for place in range(1, 11, 1):
        for kai in range(1, 7, 1):
            for day in range(1, 13, 1):
                for r in range(1, 13, 1):
                    race_id = str(year) + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
                    race_id_list.append(race_id)

    results = Results.scrape(race_id_list)
    results.to_pickle(f'{year}results.pickle')



pickleファイルに保存

In [15]:
results.to_pickle(f'{year}results.pickle')

pickleファイルのデータの読み込み

In [16]:
results = pd.read_pickle(f'{year}results.pickle')

In [ ]:
# データを一覧表示
print(results)